<a href="https://colab.research.google.com/github/Futaba-Kosuke/cat_camera_ml/blob/issue%2F%231/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install ngrok


In [15]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-11-07 08:20:50--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.54.205.131, 34.231.196.161, 52.21.51.142, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.54.205.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  54.0MB/s    in 0.2s    

2020-11-07 08:20:50 (54.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## 2. Execute ngrok

In [16]:
import time
get_ipython().system_raw('./ngrok http 3000 &')
time.sleep(1)
!curl -s http://localhost:4040/api/tunnels | python3 -c \
  "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://bd37a860f521.ngrok.io


## 3. Run API Server

### 1. 各種設定



In [2]:
!nvidia-smi

Sat Nov  7 08:20:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from flask import Flask, request, jsonify, send_file, make_response, send_from_directory

import shutil
import os, glob
import base64
import json

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from PIL import Image

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import torch
from torch import nn, optim

import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
net_names = ('densenet',)

In [6]:
# base64画像をnumpy画像に変換
def base64_to_numpy(img_base64):
    img_bytes = base64.b64decode(img_base64)
    temp = np.frombuffer(img_bytes, np.uint8)
    img_np = cv2.imdecode(temp, cv2.IMREAD_ANYCOLOR)
    return img_np

# 学習データを整形Tensorにして返す
def get_datas_and_labels(data_list):
    img_list_np = [None] * len(data_list)
    labels = [None] * len(data_list)

    for i, data in enumerate(data_list):
        img_list_np[i] = cv2.resize(base64_to_numpy(data['img'].encode('utf-8')), (200, 200))
        labels[i] = data['label']
    img_list_np = np.asarray(img_list_np)

    # X = torch.from_numpy(img_list_np.astype(np.float32))
    # y = torch.as_tensor(LabelEncoder().fit_transform(labels))
    X = img_list_np.astype(np.float32)
    y = LabelEncoder().fit_transform(labels)

    return X, y

# 層化交差検証でデータを分割
def stratified_fold(X, y, val_split=0.25):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
    return (X_train, y_train), (X_test, y_test)

# 入力データと出力ラベルからDataLoaderを生成
def get_data_loader(X, y, batch_size):
    transform = torchvision.transforms.Compose([
        transforms.Resize((200, 200)),
        transforms.RandomHorizontalFlip(p=0.5),
        torchvision.transforms.ToTensor(),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    dataset = MyDataset(X, y, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size)
    return data_loader

In [7]:
# 自分用データセットを作成するクラス
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [8]:
# 学習用関数
def train_net(net, criterion, optimizer, scheduler, num_epochs, data_loaders, target_acc=100):
    for epoch in range(num_epochs):
        print('Epoch %d/%d' % (epoch, num_epochs - 1))
        print('-' * 10)

        running_loss_sum = 0
        train_loss_sum = 0
        test_loss_sum = 0

        train_cnt = 0
        test_cnt = 0
        cnt_total = 0

        # モデルの更新
        net.train()
        for i, (inputs, labels) in enumerate(data_loaders['train']):
            inputs = inputs.to(device).cuda()
            labels = labels.to(device).cuda()
            # 勾配の初期化
            optimizer.zero_grad()
            # 予測
            outputs = net(inputs)
            # 損失の導出
            loss = criterion(outputs, labels)
            # 逆伝播
            loss.backward()
            # 勾配の更新
            optimizer.step()

            running_loss_sum += loss.item()
            train_loss_sum += loss.item()
            train_cnt += 1
            if i % 50 == 0 and not i == 0:
                print('[%d] running_loss: %.6f' % (i + 1, running_loss_sum / 50))
                running_loss_sum = 0

        # モデルの評価
        with torch.no_grad():
            net.eval()
            cnt_correct = 0
            group_cnt = [0, 0, 0]
            for i, (inputs, labels) in enumerate(data_loaders['test']):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 予測
                outputs = net(inputs)
                # 損失の導出
                loss = criterion(outputs, labels)
                # lossの加算
                test_loss_sum += loss.item()
                test_cnt += 1

                _, predicted = torch.max(outputs, 1)
                is_correct = (predicted == labels).squeeze()
                group_cnt[0] += np.count_nonzero(labels.to('cpu').detach().numpy().copy() == 0)
                group_cnt[1] += np.count_nonzero(labels.to('cpu').detach().numpy().copy() == 1)
                group_cnt[2] += np.count_nonzero(labels.to('cpu').detach().numpy().copy() == 2)
                
                for j in range(len(is_correct)):
                    cnt_correct += is_correct[j].item()  # 正解なら1, 不正解なら0
                    cnt_total += 1

            print('labels: ', group_cnt)
            print('train_loss_ave:\t%.6f' % (train_loss_sum / train_cnt))
            print('test_loss_ave:\t%.6f' % (test_loss_sum / test_cnt))
            print('test_accuracy: \t%.6f %%' % (100 * cnt_correct / (cnt_total)))

            net.train()

        scheduler.step()

        if (100 * cnt_correct / (cnt_total)) > target_acc:
            print('finish: %.6f' % (100 * cnt_correct / (cnt_total)))
            break

### 2. 学習モデルの設定

In [9]:
batch_size = 10
num_workers = 2

num_epochs = 10

In [10]:
# 学習モデルの選定
densenet = models.densenet161(pretrained=True)
last_in_features = densenet.classifier.in_features
densenet.classifier = nn.Linear(last_in_features, 10)
densenet = densenet.to(device)

nets = {
    'densenet': densenet
}

nets

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


{'densenet': DenseNet(
   (features): Sequential(
     (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu0): ReLU(inplace=True)
     (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (denseblock1): _DenseBlock(
       (denselayer1): _DenseLayer(
         (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu1): ReLU(inplace=True)
         (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu2): ReLU(inplace=True)
         (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       )
       (denselayer2): _DenseLayer(
         (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_run

In [11]:
# 損失・最適関数の定義
from torch import optim

criterion = {
    i: nn.CrossEntropyLoss()
    for i in net_names
}
optimizer = {
    i: optim.SGD(nets[i].parameters(), lr=0.001, momentum=0.9)
    for i in net_names
}
scheduler = {
    i: optim.lr_scheduler.StepLR(optimizer[i], step_size=3, gamma=0.1)  # 10エポックごとに学習率が1/10に更新
    for i in net_names
}

### 3. Flaskの起動

In [17]:
app = Flask(__name__)

@app.route('/', methods=['POST'])
def hello():
    return {'a': 'a'}

@app.route('/training', methods=['POST'])
def training():

    print('train')

    data = request.data.decode('utf-8')
    data = json.loads(data)

    data_list = data['data_list']

    # 渡されたdata_listから入力データX, 出力データyを整形
    X, y = get_datas_and_labels(data_list)

    # 層化交差検証でデータを分割
    (X_train, y_train), (X_test, y_test) = stratified_fold(X, y)

    # DataLoaderの作成
    data_loaders = {}
    data_loaders['train'] = get_data_loader(X_train, y_train, batch_size=batch_size)
    data_loaders['test']  = get_data_loader(X_test,  y_test,  batch_size=len(X_test))

    # 学習の実行
    net_name = 'densenet'
    train_net(
        net = nets[net_name],
        criterion = criterion[net_name],
        optimizer = optimizer[net_name],
        scheduler = scheduler[net_name],
        num_epochs = num_epochs,
        data_loaders = data_loaders,
    )

    model_path = 'models'
    for net_name in net_names:
        PATH = os.path.join(model_path, '{}.pth'.format(net_name))
        torch.save(nets[net_name].state_dict(), PATH)

    return '200'

if __name__ == '__main__':
    app.run(port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)


train
Epoch 0/9
----------
labels:  [4, 4, 4]
train_loss_ave:	2.481927
test_loss_ave:	1.489575
test_accuracy: 	41.666667 %
Epoch 1/9
----------
labels:  [4, 4, 4]
train_loss_ave:	1.278430
test_loss_ave:	1.114328
test_accuracy: 	41.666667 %
Epoch 2/9
----------
labels:  [4, 4, 4]
train_loss_ave:	1.111037
test_loss_ave:	1.082560
test_accuracy: 	50.000000 %
Epoch 3/9
----------
labels:  [4, 4, 4]
train_loss_ave:	1.044576
test_loss_ave:	1.014755
test_accuracy: 	50.000000 %
Epoch 4/9
----------
labels:  [4, 4, 4]
train_loss_ave:	1.035334
test_loss_ave:	1.003576
test_accuracy: 	41.666667 %
Epoch 5/9
----------
labels:  [4, 4, 4]
train_loss_ave:	1.006653
test_loss_ave:	1.001897
test_accuracy: 	50.000000 %
Epoch 6/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.919582
test_loss_ave:	0.974396
test_accuracy: 	50.000000 %
Epoch 7/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.943406
test_loss_ave:	0.981312
test_accuracy: 	50.000000 %
Epoch 8/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.

127.0.0.1 - - [07/Nov/2020 08:22:30] "POST /training HTTP/1.1" 200 -


train
Epoch 0/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.916978
test_loss_ave:	0.963819
test_accuracy: 	50.000000 %
Epoch 1/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.940405
test_loss_ave:	1.075611
test_accuracy: 	41.666667 %
Epoch 2/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.923493
test_loss_ave:	1.053624
test_accuracy: 	33.333333 %
Epoch 3/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.927496
test_loss_ave:	0.948943
test_accuracy: 	50.000000 %
Epoch 4/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.961380
test_loss_ave:	0.952646
test_accuracy: 	50.000000 %
Epoch 5/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.951427
test_loss_ave:	0.958932
test_accuracy: 	41.666667 %
Epoch 6/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.937532
test_loss_ave:	1.010064
test_accuracy: 	41.666667 %
Epoch 7/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.921168
test_loss_ave:	1.003603
test_accuracy: 	41.666667 %
Epoch 8/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.

127.0.0.1 - - [07/Nov/2020 08:25:01] "POST /training HTTP/1.1" 200 -


train
Epoch 0/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.926536
test_loss_ave:	1.003439
test_accuracy: 	50.000000 %
Epoch 1/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.945746
test_loss_ave:	0.968498
test_accuracy: 	58.333333 %
Epoch 2/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.909736
test_loss_ave:	0.943743
test_accuracy: 	58.333333 %
Epoch 3/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.942105
test_loss_ave:	0.956803
test_accuracy: 	50.000000 %
Epoch 4/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.924990
test_loss_ave:	1.000619
test_accuracy: 	50.000000 %
Epoch 5/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.936881
test_loss_ave:	1.005574
test_accuracy: 	58.333333 %
Epoch 6/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.952063
test_loss_ave:	0.974712
test_accuracy: 	41.666667 %
Epoch 7/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.938839
test_loss_ave:	0.982386
test_accuracy: 	58.333333 %
Epoch 8/9
----------
labels:  [4, 4, 4]
train_loss_ave:	0.

127.0.0.1 - - [07/Nov/2020 08:26:08] "POST /training HTTP/1.1" 200 -
